In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

import os
from datetime import datetime

from ff_app.execution import run_data_pull, update_google_sheet
from ff_app import data_prep

10:49:12 INFO:NumExpr defaulting to 8 threads.


In [2]:
week = 5

short_df = run_data_pull(week=week)
# short_df, all_df = run_data_pull(week=week, return_all_games=True)

print(short_df.shape)

10:49:12 DEBUG:Starting new HTTPS connection (1): api.the-odds-api.com:443
10:49:12 DEBUG:https://api.the-odds-api.com:443 "GET /v4/sports/americanfootball_ncaaf/odds?markets=spreads%2Ctotals&regions=us&oddsFormat=american&dateFormat=iso&api_key=ce31cd809f1458374e07be4035c3d210 HTTP/1.1" 200 15517
10:49:12 INFO:Pulled 56 total games for week 5 in 2023
10:49:12 INFO:Picks sheet data has shape (56, 11)
10:49:12 INFO:Saving game data to disk at '/Users/hoke/dev/football/game_lists/2023/week5.csv'


(56, 11)


In [3]:
short_df.head()

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Description,Spread,Total,Implied Score
0,5,09/28 07:30 PM (Thu),N,Western Kentucky Hilltoppers,Middle Tennessee Blue Raiders,Western Kentucky Hilltoppers,Middle Tennessee Blue Raiders,WKU vs MTSU,-5.5,60.0,32 - 27
1,5,09/28 07:30 PM (Thu),N,Tulsa Golden Hurricane,Temple Owls,Tulsa Golden Hurricane,Temple Owls,TUL vs TEM,-3.0,55.5,29 - 26
2,5,09/28 08:00 PM (Thu),N,Sam Houston State Bearkats,Jacksonville State Gamecocks,Jacksonville State Gamecocks,Sam Houston State Bearkats,JVILLE @ SHSU,-6.5,36.5,21 - 15
3,5,09/29 07:00 PM (Fri),N,NC State Wolfpack,Louisville Cardinals,Louisville Cardinals,NC State Wolfpack,LVILLE @ NC ST,-3.0,55.0,29 - 26
4,5,09/29 09:00 PM (Fri),N,UTEP Miners,Louisiana Tech Bulldogs,UTEP Miners,Louisiana Tech Bulldogs,UTEP vs LT,-1.0,53.0,27 - 26


In [4]:
check_team = 'Clemson'

short_df[(short_df['Favorite'].str.contains(check_team)) | (short_df['Underdog'].str.contains(check_team))]

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Description,Spread,Total,Implied Score
13,5,09/30 12:00 PM (Sat),N,Syracuse Orange,Clemson Tigers,Clemson Tigers,Syracuse Orange,CLEM @ SYR,-7.0,53.5,30 - 23


In [5]:
short_df[short_df['Mandatory'] == 'Y']

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Description,Implied Score


In [5]:
team_list = [
    'Oregon State',
    'Texas Longhorns',
    'LSU',
    'Notre Dame',
    'Bowling Green'
]

all_team_list = short_df['Favorite'].tolist() + short_df['Underdog'].tolist()
mandatory_list = []
for team in team_list:
    for t in all_team_list:
        if team in t:
            mandatory_list.append(t)

mandatory_picks = short_df[(short_df['Favorite'].isin(mandatory_list)) |
                           (short_df['Underdog'].isin(mandatory_list))]
mandatory_picks

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Description,Spread,Total,Implied Score
5,5,09/29 09:00 PM (Fri),N,Oregon State Beavers,Utah Utes,Oregon State Beavers,Utah Utes,OSU vs UTAH,-3.5,44.5,24 - 20
21,5,09/30 03:30 PM (Sat),N,Texas Longhorns,Kansas Jayhawks,Texas Longhorns,Kansas Jayhawks,TEX vs KAN,-16.5,62.5,39 - 23
29,5,09/30 03:30 PM (Sat),N,Georgia Tech Yellow Jackets,Bowling Green Falcons,Georgia Tech Yellow Jackets,Bowling Green Falcons,GT vs BGSU,-22.0,51.0,36 - 14
38,5,09/30 06:00 PM (Sat),N,Ole Miss Rebels,LSU Tigers,LSU Tigers,Ole Miss Rebels,LSU @ Ole Miss,-2.5,67.0,34 - 32
46,5,09/30 07:30 PM (Sat),N,Duke Blue Devils,Notre Dame Fighting Irish,Notre Dame Fighting Irish,Duke Blue Devils,ND @ DUKE,-5.5,52.0,28 - 23


In [6]:
short_df.loc[mandatory_picks.index.tolist(), 'Mandatory'] = 'Y'
short_df[short_df['Mandatory'] == 'Y']

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Description,Spread,Total,Implied Score
5,5,09/29 09:00 PM (Fri),Y,Oregon State Beavers,Utah Utes,Oregon State Beavers,Utah Utes,OSU vs UTAH,-3.5,44.5,24 - 20
21,5,09/30 03:30 PM (Sat),Y,Texas Longhorns,Kansas Jayhawks,Texas Longhorns,Kansas Jayhawks,TEX vs KAN,-16.5,62.5,39 - 23
29,5,09/30 03:30 PM (Sat),Y,Georgia Tech Yellow Jackets,Bowling Green Falcons,Georgia Tech Yellow Jackets,Bowling Green Falcons,GT vs BGSU,-22.0,51.0,36 - 14
38,5,09/30 06:00 PM (Sat),Y,Ole Miss Rebels,LSU Tigers,LSU Tigers,Ole Miss Rebels,LSU @ Ole Miss,-2.5,67.0,34 - 32
46,5,09/30 07:30 PM (Sat),Y,Duke Blue Devils,Notre Dame Fighting Irish,Notre Dame Fighting Irish,Duke Blue Devils,ND @ DUKE,-5.5,52.0,28 - 23


In [7]:
update_google_sheet(short_df)

10:51:41 INFO:Using credentials directory '/Users/hoke/dev/football/credentials'
10:51:41 INFO:Obtained credentails from existing 'token.json' file
10:51:41 INFO:Obtaining a token using credentials stored in 'credentials.json
10:51:41 DEBUG:Generated new state jcnn5Xqrmbz6CMdYM9BFrahscmvPyT.


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=442514865556-pp8un74st0drc1kdpurs4j8bmfav10gs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A55508%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=jcnn5Xqrmbz6CMdYM9BFrahscmvPyT&access_type=offline


10:51:50 INFO:"GET /?state=jcnn5Xqrmbz6CMdYM9BFrahscmvPyT&code=4/0AfJohXm98UgKGQ0SqxlwAyXHlz7UpwZG6XZdDmdOS6cYMQ8cBWJhBC6hhUCPUBh7clWJxg&scope=https://www.googleapis.com/auth/spreadsheets HTTP/1.1" 200 65
10:51:50 DEBUG:Encoding `client_id` "442514865556-pp8un74st0drc1kdpurs4j8bmfav10gs.apps.googleusercontent.com" with `client_secret` as Basic auth credentials.
10:51:50 DEBUG:Requesting url https://oauth2.googleapis.com/token using method POST.
10:51:50 DEBUG:Supplying headers {'Accept': 'application/json', 'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'} and data {'grant_type': 'authorization_code', 'code': '4/0AfJohXm98UgKGQ0SqxlwAyXHlz7UpwZG6XZdDmdOS6cYMQ8cBWJhBC6hhUCPUBh7clWJxg', 'redirect_uri': 'http://localhost:55508/'}
10:51:50 DEBUG:Passing through key word arguments {'timeout': None, 'auth': <requests.auth.HTTPBasicAuth object at 0x7fbaf0aea2e0>, 'verify': True, 'proxies': None}.
10:51:50 DEBUG:Starting new HTTPS connection (1): oauth2.googleapis.com:443
10:5